# 前言

将`isl`的`ir`重新生成到`c`, 算是理论联系实际中非常重要的一环, 但是目前大部分教程并没有这块内容, 导致多面体编译的上手难度非常陡峭. 因此我在学习`ppcg`的代码过程中, 决定将学会的内容进行整理, 补齐此前多面体编译教程中缺失的部分. 

为了使读者可以延续着[Polyhedral Tutorials](https://zhuanlan.zhihu.com/p/553703704)的内容继续学习, 因此继续使用`python`接口来演示. 因此需要使用到`pet/isl`中一些未导出的类和函数, 所以本教程需要配合我修改过的[pet](https://github.com/zhen8838/pet)使用. 

所有教程源码位于[这里](https://github.com/zhen8838/isl_learn).

## 关于isl接口导出

`isl`导出函数比较简单, 分为以下几步:
1. 首先需要在类/函数前加上`__isl_export`
2. 执行`make isl.py`即可更新python接口. 
   1. `make`的过程中使用了isl提供的`extract_interface`程序
      1. 它可以解析`isl`源码生成出对应的`c++/python`接口(注意部分类型导出到`cpp`时会需要特殊处理)

⚠️ `isl`在导出接口时对参数的使用做了规范化, 虽然isl部分的接口导出都是自动的, 但是`pet`的接口导出时就需要自己注意. 其中`__isl_take`表示参数指向的对象被该函数接管并且可能不再使用, 因此通常需要copy. 而`__isl_keep`则表示被当前对象暂时使用, 退出后可能会作为其他函数参数.

# C CodeGen

首先我们使用`pet`解析一个代码获取对应的`schedule tree`:

In [13]:
import pet
import isl

def parse_code(source: str, func_name: str):
  with open("/tmp/parse_code.c", "w") as f:
    f.write(source)
  scop = pet.scop.extract_from_C_source("/tmp/parse_code.c", func_name)
  context = scop.get_context()
  schedule = scop.get_schedule()
  reads = scop.get_may_reads()
  writes = scop.get_may_writes()
  return (scop, context, schedule, reads, writes)


scop, context, schedule, reads, writes = parse_code("""
void foo()
{
	int i;
	int a;

#pragma scop
	for (i = 0; i < 10; ++i)
		a = 5;
#pragma endscop
}
""", "foo")

class CSource():
  def __init__(self, path: str) -> None:
    with open(path, 'r') as f:
      self.context = f.read()
  def _repr_html_(self) -> str:
    return "<pre class='code'><code class=\"cpp hljs\">" + self.context + "</code></pre>"


## 默认的CodeGen输出

整体流程是遍历`schedule tree`build为`ast tree`, 然后再遍历`ast tree`打印为`c`代码. 其中通过各种`callback`函数处理需要添加的内容. 在不做任何修改的情况下, 打印输出为如下:

In [14]:

printer = isl.printer.from_file('/tmp/1.c')
builder = isl.ast_build()
tree: isl.ast_node = builder.node_from(schedule)
options = isl.ast_print_options.alloc()
tree.print(printer, options)
printer.flush()

CSource('/tmp/1.c')

## 使用Ast Print CallBack

其中`ast_print_options`支持两种`callback`函数, 用于在遍历`ast`的过程中, 自定义处理需要的信息:
```python
options.set_print_for(callback)
options.set_print_user(callback)
```

比如我们可以在`print_for`的时候添加基于`openmp`的并行:

In [15]:
def print_for_callback(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_for):
  # when loop can parallel execute:
  p.start_line()
  p.print_str("#pragma omp parallel for")
  p.end_line()
  node.print(p, opt)
  return p


printer = isl.printer.from_file('/tmp/2.c')
options = isl.ast_print_options.alloc()
options = options.set_print_for(print_for_callback)
tree.print(printer, options)
printer.flush()
CSource('/tmp/2.c')


为了让打印出来的代码符合`c`的形式, 可以自定义`print_user`:

In [16]:
def print_user_callback(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_user):
  p = p.start_line()
  p = p.print_str(isl.ast_expr.to_C_str(node.expr()))
  p = p.print_str(";")
  p = p.end_line()
  return p

printer = isl.printer.from_file('/tmp/3.c')
options = isl.ast_print_options.alloc()
# options = options.set_print_for(print_for_callback)
options = options.set_print_user(print_user_callback)
tree.print(printer, options)
printer.flush()
CSource('/tmp/3.c')

注意到这里输出的for循环的结果还是不符合c的格式, 我们可以修改`print_for_callback`函数, 不过`isl`中也提供了切换输出格式的功能, 即可以设定`isl printer`的`format`:

In [17]:

def print_user_callback(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_user):
  p = p.start_line()
  p = p.print_str(isl.ast_expr.to_C_str(node.expr()))
  p = p.print_str(";")
  p = p.end_line()
  return p

printer = isl.printer.from_file('/tmp/4.c')
printer.set_output_format(isl.ISL_FORMAT.C)
options = isl.ast_print_options.alloc()
# options = options.set_print_for(print_for_callback)
options = options.set_print_user(print_user_callback)
tree.print(printer, options)
printer.flush()
CSource('/tmp/4.c')

## CodeGen With Stmt

注意到上面输出的代码中每个`statement`都是用顺序编号的`id`来代替表示的, 这样也不是最终可用的格式. 我们需要将这些`id`和源代码中的实际的语句对应起来才行.

1. `ast_build`提供了`at_each_domain`的`callback`, 获取`domain`中使用到的`id`, 并查找到`pet`解析的`scop`中所对应真实的`stmt`, 并设定`annotation`记录映射关系.
2. 在`print_user`中通过`isl_ast_node_get_annotation`获取`annotation`, 以及对应的映射关系, 而后直接通过`printer`输出真实的`stmt`.

In [18]:
def find_stmt_from_scop(id: isl.id) -> pet.stmt:
  """ 在pet解析的scop中找到对应的stmt.  """
  n_stmt = scop.get_n_stmt()
  for i in range(n_stmt):
    stmt = scop.get_stmt(i)
    domain = stmt.get_domain()
    id_i = domain.get_tuple_id()
    if (id.ptr == id_i.ptr):
      return stmt
 
id_dict = dict()

def at_each_domain(node: isl.ast_node_user, build: isl.ast_build):
  expr: isl.ast_expr_op = node.get_expr()
  arg: isl.ast_expr_id = expr.get_arg(0)
  id: isl.id = arg.get_id()
  stmt: pet.stmt = find_stmt_from_scop(id)
  map = build.get_schedule().as_map()
  map = map.reverse()
  iterator_map = map.as_pw_multi_aff()

  def pullback_index(index: isl.multi_pw_aff, id: isl.id):
    return index.pullback(iterator_map)

  ref2expr = stmt.build_ast_exprs(build, pullback_index, None)
  id_dict[id.ptr] = (stmt, ref2expr)

  return node.set_annotation(id)


def print_user(p: isl.printer, opt: isl.ast_print_options, node: isl.ast_node_user):
  # when loop can parallel execute:
  id = node.annotation()
  (stmt, ref2expr) = id_dict[id.ptr]
  stmt: pet.stmt
  p = stmt.print_body(p, ref2expr)
  return p


printer = isl.printer.from_file('/tmp/5.c')
printer.set_output_format(isl.ISL_FORMAT.C)
builder = isl.ast_build()
builder = builder.set_at_each_domain(at_each_domain)
tree: isl.ast_node = builder.node_from(schedule)
options = isl.ast_print_options.alloc()
options = options.set_print_user(print_user)
tree.print(printer, options)
printer.flush()

CSource('/tmp/5.c')
